# Substance Abuse Project

To be able to predict the outcome of substance abuse treatment is a high value solution. A model being able to make the distinction between a passing and failing case is difficult and requires alot of information. For this iteration, we utlized the TEDs 2018 dataset provided by the **_Center for Behavioral Health Statistics and Quality_** and the **_Substance Abuse and Mental Health Services Administration_**

---


## Installing Dependencies
If necessary please uncomment the following lines and install the dependencies with pip. Our notebook uses each one of these and it's important that all can be imported to avoid errors.

In [ ]:
# !pip install sagemaker
# !pip install matplotlib
# !pip install boto3
# !pip install pandas
# !pip install fsspec
# !pip install s3fs
# !pip install sklearn
# !pip install seaborn
# !pip install progress

## Transforming Data
For this section we work on removing uninmportant data, unlabelled rows, and modifying our dataset to be used with the XGBoost training algorithm.
### Importing Dependencies and Setting Environment Variables
In this next code block we import boto3 and pandas. Pandas will be important for our first step, importing the CSV file as a **dataframe** a sort of programatically accessible table that we can use in our python code. boto3 will become important in a later step when we need to export our split data back to s3. Though there are other popular dataframe modules such as dask, pandas remains the most open source with the best source documentation and clear instruction. It also features the fastest operation time. We also import math, json, numpy and matplotlib.pyplot for later use. Below that we set our environment variables which we use throughout the notebook.

We also save our important filenames and the name of our bucket to the notebook for use later. Putting all this data together in one step means avoiding running extraneous code when rerunning code in later sequences

In [1]:
import boto3, pandas as pd, math, json, numpy as np, matplotlib.pyplot as plt, os

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

bucket = 'bucket-sagemaker-substance-abuse'
raw_file_2017 = 'data/raw/tedsd_puf_2017.csv'
raw_file_2018 = 'data/raw/tedsd_puf_2018.csv'
train_subfolder = 'data/train/'
eval_subfolder = 'data/evaluation/'
label = "REASON" # Desired Label
objective = 'binary:logistic' # binary:logistic or multi:softmax
training_rounds = 100 # 1000-1500 for production, 10-100 for testing and development

### Saving our CSV as a pandas Dataframe Object
Most of the lines in this block format a string for use with pandas, telling it where our file is located. I've opted for *join concatenation with a slash* instead of operator concatenation which uses more memory and is less efficient. The latter half of this block of code reads out the data from our source file to the pandas dataframe which we can mutate during our transformations. Any dataframes method with a possible `inplace` attribute should be set to true so that the dataframe is mutated and not returned except for in the case that you're saving it to a variable. 

This step may take a few seconds

In [ ]:
raw_location_2018 = 's3://{}/{}'.format(bucket, raw_file_2018)
raw_location_2017 = 's3://{}/{}'.format(bucket, raw_file_2017)
df_2018 = pd.read_csv(raw_location_2018)
print(f"2018 Dataframe Loaded: { len(df_2018) } rows x {len(df_2018.columns)} columns")
df_2017 = pd.read_csv(raw_location_2017)
print(f"2017 Dataframe Loaded: { len(df_2017) } rows x {len(df_2017.columns)} columns")
df = pd.concat([df_2018, df_2017])
print(f"Dataframe Loaded: { len(df) } rows x {len(df.columns)} columns")

### Removing Extraneous Columns
In the next step we take the names of all the columns we want to remove and set them in an array. Originally we would use an array to loop through and drop them one by one however pandas `drop` function can take *list-like* parameters and so we can actually save both time and resources by doing it all at once. This will significantly increase development speeds

In [ ]:
remove = ["CASEID", "EMPLOY_D", "DETNLF_D", "LIVARAG_D", "ARRESTS_D", "SERVICES_D", "SUB1_D", "FREQ1_D", "SUB2_D", "FREQ2_D", "SUB3_D", "FREQ3_D", "FREQ_ATND_SELF_HELP_D", "LOS", "DISYR"]
data_top = df.columns
    
# display 
print(data_top)
index_names = df[ (df['REASON'] == 3) | (df['REASON'] == 5) | (df['REASON'] == 6) | (df['REASON'] == 7)].index # Collects Death, Incarceration, Termination, and Other labeled rows for removal
df.drop(index_names, inplace = True) # Drops rows grabbed above
df.drop(remove, inplace=True, axis=1) # Drops extraneous columns in 'remove' array above
df.dropna(axis=0, subset=[label], inplace=True) # Drops rows missing desired label
print(f"Dataframe Transformed: { len(df) } rows x {len(df.columns)} columns")

### Transforming Categorical Data to Binary Columns
<span style="color:red">**BUG**</span> For this step we transform our categorical data into binary columns. To do this we call `get_dummies` on each column listed in the transform array as we loop through it. We then take that *dummy* dataframe and we concatenate it with the current dataframe along the vertical axis. After we loop through the entire array, we drop all the coulmns in the transform array as they're now redundant. Because there is a bug in this step, probably caused by a memory shortage or a CPU resource shortage, we should think about creating an empty dataframe in which we concatenate all dummies together and then concatenate that at the end instead of many times during the loop. In the last update I removed "Reason" from the transform list as we'll need to know what that becomes

In [ ]:
transform = ["AGE", "GENDER", "RACE", "ETHNIC", "MARSTAT", "EDUC", "EMPLOY", "DETNLF", "PREG", "VET", "LIVARAG", "PRIMINC", "ARRESTS", "SERVICES", "METHUSE", "DAYWAIT", "PSOURCE", "DETCRIM", "NOPRIOR", "SUB1", "ROUTE1", "FREQ1", "FRSTUSE1", "SUB2", "ROUTE2", "FREQ2", "FRSTUSE2", "SUB3", "ROUTE3", "FREQ3", "FRSTUSE3", "IDU", "DIVISION", "REGION", "STFIPS"];
df = pd.get_dummies(df, prefix=transform, columns=transform)
print(f"Dataframe Transformed: { len(df) } rows x {len(df.columns)} columns")

### Convert to Binary Classification and Modify Data for Algorithm
This code moves our label to the front of the dataframe. It also decrements 1 from the Reason category so the the XDGBoost Algorithm can understand the classes. Our current categories are:
- **0**: Positive Result
- **1**: Non-positive Result

For reasons having to do with confidence, we've phased out multi classification, though it can be reenabled by modifying the hyperparameters for the algorithm and modifying replace statements below. THe replace methods called may seem redundant but they are created in a way so that commenting out certain statements allows us to roll back to multi.

In [ ]:
df[label] = df[label].replace([3, 4, 2], [0, 0, 0]) # Converts to Binary
column = df.pop(label)
df.insert(0, label, column)
class_pool = pd.get_dummies(df[label]) # For Debugging
print(f"Labels altered, class pool reduced to {len(class_pool.columns)}")

## Split Data
To do proper model training we'll use a holdout dataset for evaluation and so we'll need to split the set into 80% and 20%
### Separating Training and Evaluation Sets and Uploading them to S3
To complete our tranformation section, we grab a random sample of our data that makes up half of our data and we transport it to S3 utilizing a boto3 s3 client after converting the new dataframe to a CSV. If the command is run twice it will overwrite previous data. This section also creates an Evaluation Dataframe and ships it in a CSV to a seperate folder from the training data.

This process will take up to 3 minutes

In [ ]:
sdf = df.sample(frac=1)
print(f"Random Dataframe Created: { len(sdf) } rows x {len(sdf.columns)} columns")
training_end_index = math.floor(len(sdf)*0.8)
eval_start_index = training_end_index + 1
end = len(sdf) - 1
tdf = sdf.iloc[0:training_end_index] # first five rows of dataframe
print(f"Training Dataframe Created: { len(tdf) } rows x {len(tdf.columns)} columns")
edf = sdf.iloc[training_end_index:end]
print(f"Evaluation Dataframe Created: { len(edf) } rows x {len(edf.columns)} columns")

t_filename = 'training_data.csv'
e_filename = 'evaluation_data.csv'
tdf.to_csv(t_filename, header=False, index=False)
edf.to_csv(e_filename, header=False, index=False)
s3 = boto3.resource('s3')
s3.meta.client.upload_file(t_filename, bucket, train_subfolder + t_filename)
s3.meta.client.upload_file(e_filename, bucket, eval_subfolder + e_filename)
print(f"Operation Complete, Files Uploaded")



In [ ]:
print(f"deleting local files")
os.remove(t_filename)
os.remove(e_filename)
print(f"local files deleted")

###### (With the exception of block 1, the following can be independant)
***
## Testing The Model's Hyperparameters

### Setting Sagemaker Environment
Now we get to important stuff. Here we leave the transformation phase and move into the training phase. This block saves our region into the notebook making it usable in our model. The steps below do not need to be run in conjunction with those above, provided you've already completed the above steps

### Setting Model Parameters and HyperParameters
In this section we set up our XDGBoost training and pass it our basic values and hyperparameters. `num_round` determines how many rounds of training the model goes through. `num_class` alters how many classes it accepts. If the objective changes to binary this can be removed. Using A/B Testing I was able to discover there's not really a point to doing more than *1000* rounds of training. The training m-error may keep dropping but the evaluation m-error stays roughly stagnant

In [2]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput

model_bucket_prefix = 'output/models'

s3_output_location = 's3://{}/{}/{}'.format(bucket, model_bucket_prefix, 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role = 'arn:aws:iam::620700586481:role/sagemaker-substance-abuse-project-role',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[Rule.sagemaker(rule_configs.create_xgboost_report())]
)

xgb_model.set_hyperparameters(
    eval_metric="auc",
    max_depth = 6,
    min_child_weight = 1,
    rate_drop=0.3,
    objective = objective
)
objective_metric_name = "validation:auc"

prefix = "data"

train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train/training_data.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "evaluation/evaluation_data.csv"), content_type="csv"
)

AWS Region: us-east-2
257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.2-1


### Using Hyperparameter Tuning
For this part we'll begin hyperparameter tuning. For reference see [https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/xgboost_random_log/hpo_xgboost_random_log.ipynb](https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/xgboost_random_log/hpo_xgboost_random_log.ipynb). We'll start with __Automatic Scaling__ and continue from there.

In [ ]:
print("Declaring Hyperparameter ranges")

hyperparameter_ranges = {
    "alpha": ContinuousParameter(0.01, 10, scaling_type="Auto"),
    "lambda": ContinuousParameter(0.01, 10, scaling_type="Auto"),
    "min_child_weight": ContinuousParameter(0.01, 120, scaling_type="Auto"),
    "subsample": ContinuousParameter(0.5, 1, scaling_type="Auto"),
    "eta": ContinuousParameter(0.1, 0.5, scaling_type="Auto"),
    "num_round": IntegerParameter(1, 4000, scaling_type="Auto")
}


tuner_auto = HyperparameterTuner(
    xgb_model,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=30,
    max_parallel_jobs=3,
    strategy="Random"
)

print("Starting Automatic Tuning")

tuner_auto.fit(
    {"train": train_input, "validation": validation_input}, 
    wait=True,
    include_cls_metadata=False
)

boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner_auto.latest_tuning_job.job_name
)["HyperParameterTuningJobStatus"]

print("Automatic Tuning Complete")

Declaring Hyperparameter ranges
Starting Automatic Tuning
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Analyzing Tuning Job Results and Training New Model

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# check jobs have finished
auto_output = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner_auto.latest_tuning_job.job_name
)
best_training_job = boto3.client("sagemaker").describe_training_job(
    TrainingJobName=auto_output["BestTrainingJob"]["TrainingJobName"]
)

status_auto = auto_output["HyperParameterTuningJobStatus"]

assert status_auto == "Completed", "First must be completed, was {}".format(status_auto)

df_auto = sagemaker.HyperparameterTuningJobAnalytics(
    tuner_auto.latest_tuning_job.job_name
).dataframe()
df_auto["scaling"] = "auto"

g = sns.FacetGrid(df_auto, col="scaling", palette="viridis")
g = g.map(plt.scatter, "alpha", "lambda", alpha=0.6)

# Set Best Hyperparameters

best_hyperparams = best_training_job["HyperParameters"]
print("Best Accuracy: " + str(best_training_job["FinalMetricDataList"][0]["Value"]))

## Deploy Production Level Endpoint

In [ ]:
from sagemaker.serializers import CSVSerializer
predictor = tuner_auto.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge', 
    serializer=CSVSerializer()
)

In [ ]:
import numpy as np
from time import sleep
from progress.bar import Bar

print("Importing NumPy")
def predict(data, rows=1000):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    with Bar('Loading', fill='@', suffix='%(percent).1f%% - %(eta)ds') as bar:
        for array in split_array:
            predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])
            bar.next()
    return np.fromstring(predictions[1:], sep=',')

print("Created Prediction Definition")

import matplotlib.pyplot as plt

print("Imported MatPlotLib.pyplot")

prefix = "data"

raw_location = "s3://{}/{}/{}".format(bucket, prefix, "evaluation/evaluation_data.csv")
test = pd.read_csv(raw_location)

print("Created test dataframe")

print("Started Prediction")
predictions=predict(test.to_numpy()[:,1:])

plt.hist(predictions)
plt.show()

import sklearn
from sklearn import *

cutoff=0.5
print(sklearn.metrics.confusion_matrix(test.iloc[:, 0], np.where(predictions > cutoff, 1, 0)))
print(sklearn.metrics.classification_report(test.iloc[:, 0], np.where(predictions > cutoff, 1, 0)))

import matplotlib.pyplot as plt

cutoffs = np.arange(0.01, 1, 0.01)
log_loss = []
for c in cutoffs:
    log_loss.append(
        sklearn.metrics.log_loss(test.iloc[:, 0], np.where(predictions > c, 1, 0))
    )

plt.figure(figsize=(15,10))
plt.plot(cutoffs, log_loss)
plt.xlabel("Cutoff")
plt.ylabel("Log loss")
plt.show()

print(
    'Log loss is minimized at a cutoff of ', cutoffs[np.argmin(log_loss)], 
    ', and the log loss value at the minimum is ', np.min(log_loss)
)

## Create Dev Model For XGBoost Report

In [ ]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput

prefix = "output/models"

s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role = 'arn:aws:iam::620700586481:role/sagemaker-substance-abuse-project-role',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[Rule.sagemaker(rule_configs.create_xgboost_report())]
)

rounds = best_hyperparams["num_round"]

print(f"Objective will attempt to train with { rounds } training rounds expect a delay of {int((((27/100)*60) * int(rounds))/60)} minutes")

xgb_model.set_hyperparameters(
    max_depth = best_hyperparams["max_depth"],
    eta = best_hyperparams["eta"],
    gamma = 4,
    min_child_weight = best_hyperparams["min_child_weight"],
    subsample = best_hyperparams["subsample"],
    objective = best_hyperparams["objective"],
    num_round = best_hyperparams["num_round"],
    alpha = best_hyperparams["alpha"],
    reg_lambda = best_hyperparams["lambda"],
    rate_drop = best_hyperparams["rate_drop"]
)

xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)

In [ ]:
print(vars(xgb_model))

rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive
from IPython.display import FileLink, FileLinks
display("Click link below to view the XGBoost Training report", FileLink("CreateXgboostReport/xgboost_report.html"))

profiler_report_name = [rule["RuleConfigurationName"] 
                        for rule in xgb_model.latest_training_job.rule_job_summary() 
                        if "Profiler" in rule["RuleConfigurationName"]][0]
profiler_report_name
display("Click link below to view the profiler report", FileLink(profiler_report_name+"/profiler-output/profiler-report.html"))

xgb_model.model_data

### Project Goals
- [x] Get data into sagemaker
- [x] Clean Data
- [x] Transform Data
- [x] Split Data
- [x] Train Model
- [x] Modify Patterns and remove rows with now extraneous labels
- [x] Reach 80-85% Accuracy over the next 3 days
- [x] Deploy Model
- [ ] Create Demo?